#### Defino words_to_numbers

In [1]:
words_to_numbers = {
    "veintiuno": 21,
    "veintidós": 22,
    "veintitres": 23,
    "veinticuatro": 24,
    "veinticinco": 25,
    "veintiseis": 26,
    "veintisiete": 27,
    "veintiocho": 28,
    "veintinueve": 29,
    "treinta": 30,
    "treinta y uno": 31,
    "treinta y dos": 32,
    "treinta y tres": 33,
    "treinta y cuatro": 34,
    "treinta y cinco": 35,
    "treinta y seis": 36,
    "treinta y siete": 37,
    "treinta y ocho": 38,
    "treinta y nueve": 39,
    "cuarenta": 40,
    "cuarenta y uno": 41,
    "cuarenta y dos": 42,
    "cuarenta y tres": 43,
    "cuarenta y cuatro": 44,
    "cuarenta y seis": 46,
    "cuarenta y siete": 47,
    "cuarenta y ocho": 48,
    "cuarenta y nueve": 49,
    "cincuenta": 50,
    "cincuenta y uno": 51,
    "cincuenta y dos": 52,
    "cincuenta y tres": 53,
    "cincuenta y cuatro": 54,
    "cincuenta y cinco": 55,
    "cincuenta y seis": 56,
    "cincuenta y siete": 57,
    "cincuenta y ocho": 58,
    "cincuenta y nueve": 59,
    "sesenta": 60,
    "sesenta y uno": 61,
    "sesenta y dos": 62,
    "sesenta y tres": 63,
    "sesenta y cuatro": 64,
    "sesenta y cinco": 65,
    "sesenta y seis": 66,
    "sesenta y siete": 67,
    "sesenta y ocho": 68,
    "sesenta y nueve": 69
}

####  Setup

In [ ]:
import pandas as pd
import os
import math
import json


In [ ]:
# Más automatizado, por si tenemos muchos archivos:

# dfs = {}
# ejecuciones = list(map(lambda x: x[:-4], os.listdir('ejecuciones')))
# for e in participantes:
#     dfs[e] = pd.read_csv(f'ejecuciones/{e}.csv')

In [ ]:
try:
  df = pd.read_csv('./raw/18.csv') # bloque 1 y 2 con dígitos, 3 y 4 con palabras.
except FileNotFoundError:
  raise Exception('Hay que agregar una carpeta llamada "ejecuciones" con los csv y nombrar a cada archivo 1, 2, 3...')

In [ ]:
df.columns #13

In [ ]:
ID = f"{df['ip'][0]}_{df['recorded_at'][0][:10]}_{json.loads(df.iloc[8, 25])['gender']}_{json.loads(df.iloc[8, 25])['hand']}"

In [ ]:
# show 'stimulus', 'response' of df
df[['stimulus', 'response']].head(20)

Les saco las tasks de entrenamiento e instrucciones y dejo sólo las que tienen algún valor en "block", que son las tasks del experimento.
Luego las ordeno primero por bloque y después por ronda, de manera ascendente.

In [ ]:
columnas_importantes = ['block', 'round', 'task', 'trial_duration', 'delay', 'audioFile', 'number', 'stimulus', 'response',  'rt']
df = df[columnas_importantes]
df = df.dropna(subset=['block'])
df = df.sort_values(by=['block', 'round'], ascending=[True, True])
# Group by 'block' and 'round', then aggregate 'task' using join
df['type'] = df.groupby(['block', 'round'])['task'].transform(lambda x: '+'.join(x))

In [ ]:
df['type'].unique()

In [ ]:
# add new ['response_1','rt_1','response_2','rt_2'] none columns
df.loc[:, 'response_1'] = None
df.loc[:, 'rt_1'] = None
df.loc[:, 'response_2'] = None
df.loc[:, 'rt_2'] = None

## Tipo 1

In [ ]:
_df = df[df['type'] == 'tone_number_task+empty_block+empty_block']

In [ ]:
indices_to_drop = []

for index, row in _df.iterrows():
    if row['task'] == 'tone_number_task' and _df.loc[index + 1, 'response'] in ['a', 's'] and _df.loc[index + 2, 'response'] in ['j', 'k']:
        _df.loc[index, 'response_1'] = _df.loc[index + 1, 'response']
        _df.loc[index, 'rt_1'] = _df.loc[index, 'trial_duration'] + _df.loc[index + 1, 'rt']
        _df.loc[index, 'response_2'] = _df.loc[index + 2, 'response']
        _df.loc[index, 'rt_2'] = 150 + _df.loc[index + 1, 'rt'] + _df.loc[index + 2, 'rt']
    else:
        indices_to_drop.append(index)

# After the loop, drop the rows using the list of indices
_df = _df.drop(indices_to_drop)

In [ ]:
_df

## Tipo 2

In [ ]:
_df = df[df['type'] == 'tone_number_task+delay_block+tone_number_task_remaining_before_delay+empty_block']

In [ ]:
indices_to_drop = []

for index, row in _df.iterrows():
    if row['task'] == 'tone_number_task' and _df.loc[index, 'response'] in ['a', 's'] and _df.loc[index + 3, 'response'] in ['j', 'k']:
        _df.loc[index, 'response_1'] = _df.loc[index, 'response']
        _df.loc[index, 'rt_1'] = _df.loc[index, 'rt']
        _df.loc[index, 'response_2'] = _df.loc[index + 3, 'response']
        _df.loc[index, 'rt_2'] = 150 + _df.loc[index + 3, 'rt']
    else:
        indices_to_drop.append(index)

# After the loop, drop the rows using the list of indices
_df = _df.drop(indices_to_drop)

In [ ]:
_df

## Tipo 3

In [ ]:
_df = df[df['type'] == 'tone_number_task+tone_number_task_remaining_after_delay+empty_block']

In [ ]:
indices_to_drop = []

for index, row in _df.iterrows():
    if row['task'] == 'tone_number_task' and _df.loc[index, 'response'] in ['a', 's'] and _df.loc[index + 2, 'response'] in ['j', 'k']:
        _df.loc[index, 'response_1'] = _df.loc[index, 'response'] 
        _df.loc[index, 'rt_1'] = _df.loc[index, 'rt']
        _df.loc[index, 'response_2'] = _df.loc[index + 2, 'response']
        _df.loc[index, 'rt_2'] = 150 + _df.loc[index + 2, 'rt']
    else:
        indices_to_drop.append(index)

# After the loop, drop the rows using the list of indices
_df = _df.drop(indices_to_drop)

In [ ]:
_df

## Tipo 4

In [ ]:
_df = df[df['type'] == 'tone_number_task+delay_block+tone_number_task_remaining_before_delay']

In [ ]:
indices_to_drop = []

for index, row in _df.iterrows():
    if row['task'] == 'tone_number_task' and _df.loc[index, 'response'] in ['a', 's'] and _df.loc[index + 2, 'response'] in ['j', 'k']:
        _df.loc[index, 'response_1'] = _df.loc[index, 'response'] 
        _df.loc[index, 'rt_1'] = _df.loc[index, 'rt']
        _df.loc[index, 'response_2'] = _df.loc[index + 2, 'response']
        _df.loc[index, 'rt_2'] = _df.loc[index + 2, 'rt']
    else:
        indices_to_drop.append(index)

# After the loop, drop the rows using the list of indices
_df = _df.drop(indices_to_drop)

In [227]:
# add new ['response_1','rt_1','response_2','rt_2'] none columns
df.loc[:, 'response_1'] = None
df.loc[:, 'rt_1'] = None
df.loc[:, 'response_2'] = None
df.loc[:, 'rt_2'] = None

## Tipo 1

In [230]:
_df = df[df['type'] == 'tone_number_task+empty_block+empty_block']

In [232]:
indices_to_drop = []

for index, row in _df.iterrows():
    if row['task'] == 'tone_number_task' and _df.loc[index + 1, 'response'] in ['a', 's'] and _df.loc[index + 2, 'response'] in ['j', 'k']:
        _df.loc[index, 'response_1'] = _df.loc[index + 1, 'response']
        _df.loc[index, 'rt_1'] = _df.loc[index, 'trial_duration'] + _df.loc[index + 1, 'rt']
        _df.loc[index, 'response_2'] = _df.loc[index + 2, 'response']
        _df.loc[index, 'rt_2'] = 150 + _df.loc[index + 1, 'rt'] + _df.loc[index + 2, 'rt']
    else:
        indices_to_drop.append(index)

# After the loop, drop the rows using the list of indices
_df = _df.drop(indices_to_drop)

In [233]:
_df

,block,round,task,trial_duration,delay,audioFile,number,stimulus,response,rt,type,response_1,rt_1,response_2,rt_2
406,0.0,1.0,tone_number_task,150.0,0.0,tones/440hz_short.mp3,68,tones/440hz_short.mp3,NaN,NaN,tone_number_task+empty_block+empty_block,a,1008.0,k,1284.0
410,0.0,2.0,tone_number_task,150.0,0.0,tones/880hz_short.mp3,67,tones/880hz_short.mp3,NaN,NaN,tone_number_task+empty_block+empty_block,s,912.0,k,1379.0
414,0.0,3.0,tone_number_task,150.0,0.0,tones/880hz_short.mp3,38,tones/880hz_short.mp3,NaN,NaN,tone_number_task+empty_block+empty_block,s,986.0,j,1164.0
422,0.0,5.0,tone_number_task,252.5,102.5,tones/880hz_short.mp3,48,tones/880hz_short.mp3,NaN,NaN,tone_number_task+empty_block+empty_block,s,903.5,j,1354.0
426,0.0,6.0,tone_number_task,252.5,102.5,tones/440hz_short.mp3,53,tones/440hz_short.mp3,NaN,NaN,tone_number_task+empty_block+empty_block,a,753.5,k,908.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,3.0,32.0,tone_number_task,150.0,0.0,tones/880hz_short.mp3,treinta,tones/880hz_short.mp3,NaN,NaN,tone_number_task+empty_block+empty_block,s,916.0,j,1184.0
367,3.0,33.0,tone_number_task,150.0,0.0,tones/880hz_short.mp3,cincuenta y seis,tones/880hz_short.mp3,NaN,NaN,tone_number_task+empty_block+empty_block,s,994.0,k,1130.0
371,3.0,34.0,tone_number_task,252.5,102.5,tones/880hz_short.mp3,cincuenta y cuatro,tones/880hz_short.mp3,NaN,NaN,tone_number_task+empty_block+empty_block,s,1086.5,k,1083.0
385,3.0,37.0,tone_number_task,252.5,102.5,tones/880hz_short.mp3,sesenta y nueve,tones/880hz_short.mp3,NaN,NaN,tone_number_task+empty_block+empty_block,s,825.5,k,823.0


## Tipo 2

In [234]:
_df = df[df['type'] == 'tone_number_task+delay_block+tone_number_task_remaining_before_delay+empty_block']

In [235]:
indices_to_drop = []

for index, row in _df.iterrows():
    if row['task'] == 'tone_number_task' and _df.loc[index, 'response'] in ['a', 's'] and _df.loc[index + 3, 'response'] in ['j', 'k']:
        _df.loc[index, 'response_1'] = _df.loc[index, 'response']
        _df.loc[index, 'rt_1'] = _df.loc[index, 'rt']
        _df.loc[index, 'response_2'] = _df.loc[index + 3, 'response']
        _df.loc[index, 'rt_2'] = 150 + _df.loc[index + 3, 'rt']
    else:
        indices_to_drop.append(index)

# After the loop, drop the rows using the list of indices
_df = _df.drop(indices_to_drop)

In [236]:
_df

,block,round,task,trial_duration,delay,audioFile,number,stimulus,response,rt,type,response_1,rt_1,response_2,rt_2
401,0.0,0.0,tone_number_task,1175.0,1025.0,tones/880hz_short.mp3,56,tones/880hz_short.mp3,s,810.0,tone_number_task+delay_block+tone_number_task_...,s,810.0,k,1174.0
430,0.0,7.0,tone_number_task,1175.0,1025.0,tones/880hz_short.mp3,43,tones/880hz_short.mp3,s,679.0,tone_number_task+delay_block+tone_number_task_...,s,679.0,j,404.0
439,0.0,9.0,tone_number_task,1142.5,992.5,tones/440hz_short.mp3,39,tones/440hz_short.mp3,a,700.0,tone_number_task+delay_block+tone_number_task_...,a,700.0,j,391.0
456,0.0,13.0,tone_number_task,1175.0,1025.0,tones/440hz_short.mp3,50,tones/440hz_short.mp3,a,691.0,tone_number_task+delay_block+tone_number_task_...,a,691.0,k,414.0
461,0.0,14.0,tone_number_task,1142.5,992.5,tones/440hz_short.mp3,25,tones/440hz_short.mp3,a,700.0,tone_number_task+delay_block+tone_number_task_...,a,700.0,j,434.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,3.0,28.0,tone_number_task,1175.0,1025.0,tones/440hz_short.mp3,cincuenta y siete,tones/440hz_short.mp3,a,572.0,tone_number_task+delay_block+tone_number_task_...,a,572.0,k,489.0
358,3.0,31.0,tone_number_task,1175.0,1025.0,tones/880hz_short.mp3,veinticuatro,tones/880hz_short.mp3,s,540.0,tone_number_task+delay_block+tone_number_task_...,s,540.0,j,481.0
375,3.0,35.0,tone_number_task,1142.5,992.5,tones/440hz_short.mp3,treinta y siete,tones/440hz_short.mp3,a,670.0,tone_number_task+delay_block+tone_number_task_...,a,670.0,j,573.0
380,3.0,36.0,tone_number_task,1175.0,1025.0,tones/440hz_short.mp3,cuarenta y tres,tones/440hz_short.mp3,a,830.0,tone_number_task+delay_block+tone_number_task_...,a,830.0,j,1062.0


## Tipo 3

In [237]:
_df = df[df['type'] == 'tone_number_task+tone_number_task_remaining_after_delay+empty_block']

In [238]:
indices_to_drop = []

for index, row in _df.iterrows():
    if row['task'] == 'tone_number_task' and _df.loc[index, 'response'] in ['a', 's'] and _df.loc[index + 2, 'response'] in ['j', 'k']:
        _df.loc[index, 'response_1'] = _df.loc[index, 'response'] 
        _df.loc[index, 'rt_1'] = _df.loc[index, 'rt']
        _df.loc[index, 'response_2'] = _df.loc[index + 2, 'response']
        _df.loc[index, 'rt_2'] = 150 + _df.loc[index + 2, 'rt']
    else:
        indices_to_drop.append(index)

# After the loop, drop the rows using the list of indices
_df = _df.drop(indices_to_drop)

In [239]:
_df

,block,round,task,trial_duration,delay,audioFile,number,stimulus,response,rt,type,response_1,rt_1,response_2,rt_2
418,0.0,4.0,tone_number_task,1142.5,992.5,tones/880hz_short.mp3,56,tones/880hz_short.mp3,s,1051.0,tone_number_task+tone_number_task_remaining_af...,s,1051.0,k,991.0
699,1.0,26.0,tone_number_task,1142.5,992.5,tones/440hz_short.mp3,68,tones/440hz_short.mp3,a,1030.0,tone_number_task+tone_number_task_remaining_af...,a,1030.0,k,555.0
193,2.0,35.0,tone_number_task,1175.0,1025.0,tones/440hz_short.mp3,treinta,tones/440hz_short.mp3,a,1071.0,tone_number_task+tone_number_task_remaining_af...,a,1071.0,k,657.0


## Tipo 4

In [240]:
_df = df[df['type'] == 'tone_number_task+delay_block+tone_number_task_remaining_before_delay']

In [241]:
indices_to_drop = []

for index, row in _df.iterrows():
    if row['task'] == 'tone_number_task' and _df.loc[index, 'response'] in ['a', 's'] and _df.loc[index + 2, 'response'] in ['j', 'k']:
        _df.loc[index, 'response_1'] = _df.loc[index, 'response'] 
        _df.loc[index, 'rt_1'] = _df.loc[index, 'rt']
        _df.loc[index, 'response_2'] = _df.loc[index + 2, 'response']
        _df.loc[index, 'rt_2'] = _df.loc[index + 2, 'rt']
    else:
        indices_to_drop.append(index)

# After the loop, drop the rows using the list of indices
_df = _df.drop(indices_to_drop)

### Análisis Respuestas Correctas / Incorrectas
Útil pa

In [ ]:
def find_first_response(responses):
    for r in responses:
        if type(r) == str:
            return r
    return None

def find_second_response(responses):
    found_first = False
    for r in responses:
        if type(r) == str:
            if found_first:
                return r
            else:
                found_first = True
    return None

In [ ]:
def get_number_of_round(rows_of_round):
    return rows_of_round.iloc[0]['number']

def get_audio_of_round(rows_of_round):
    return rows_of_round.iloc[0]['audioFile']

In [ ]:
def audio_response_matches(audio_response, audio_stimulus):
    return (audio_response == 'a' and audio_stimulus == 'tones/440hz_short.mp3') or (audio_response == 's' and audio_stimulus == 'tones/880hz_short.mp3')

def number_response_matches(number_response, number_stimulus):
    try:
        number_stimulus = int(number_stimulus)
    except ValueError:
        number_stimulus = words_to_numbers[number_stimulus]

    return (number_response == 'j' and number_stimulus < 45) or (number_response == 'k' and number_stimulus > 45)


def responses_match_with_stimulus(audio_response, number_response, audio, number):
    audio = audio_response_matches(audio_response, audio)
    number = number_response_matches(number_response, number)
    return audio and number

In [ ]:
def responded_correcty_in_block_and_round(df, block, round):
    rows_of_that_round = df[(df['block'] == block) & (df['round'] == round)]
    number = get_number_of_round(rows_of_that_round)
    audio = get_audio_of_round(rows_of_that_round)
    responses = []
    for row in rows_of_that_round.itertuples(index=False):
        responses.append(row.response)

    audio_response = find_first_response(responses)
    number_response = find_second_response(responses)
    if audio_response == None or number_response == None:
        return False

    return responses_match_with_stimulus(audio_response, number_response, audio, number)

In [ ]:
def responses_results_for_df(df):
    """
    Retorna un arreglo de arreglos de booleanos, indicando para cada muestra si respondieron bien a ambos estímulos o no.
    El primer índice del arreglo que se retorna indica el bloque [0-3] y el segundo el número de ronda de ese bloque [0-39]
    """
    responded_correctly = [[None for _ in range(40)] for _ in range(4)]
    for block_id in range(0, 4):
        for round_id in range(0, 40):
            responded_correctly[block_id][round_id] = responded_correcty_in_block_and_round(df, block_id, round_id)
    return responded_correctly

In [ ]:
df_responses = responses_results_for_df(df)

In [ ]:
# Calculate the ratio of True values
total_elements = 4 * 40
total_true = sum(row.count(True) for row in df_responses)

ratio = total_true / total_elements

# Print the ratio
print(total_elements)
print(total_true)
print(f"Ratio: {ratio:.2%}")

In [ ]:
# Las rondas que corresponden a esas 10 respuestas incorrectas habría que sacarlas de la df.
# Podemos saber cuáles son encontrando los índices de los valores que son False en df_responses.

###  Análisis Tiempos de Respuesta
Acordarse primero de filtrar (eliminar) todos los que dieron false en responses_results_for_df.

Además, falta ver qué tipo de estímulo es el número (dígito o palabra). Eso se puede ver separando los bloques.

In [ ]:
# Sabemos que tenemos 4 bloques, de 40 rondas cada uno.

# Cada una de esas rondas puede tener 3 ó 4 tareas. ¿Pueden tener 2 también? Creo que sí.
# Fijarse eso por las dudas con groupby(['block', 'round']).size().value_counts().

In [ ]:
def get_delay_of_round(rows_of_round):
    return rows_of_round.iloc[0]['delay']

In [ ]:
def get_responses_time_of_block_and_round(df, block, round):

    response_time_audio, response_time_number = None, None
    first_task, second_task, third_task, fourth_task = None, None, None, None

    rows_of_that_round = df[(df['block'] == block) & (df['round'] == round)]

    # Si esas rondas fueron descartadas por respuestas incorrectas, no hay nada que hacer.
    if rows_of_that_round.empty:
        return None, None

    delay = get_delay_of_round(rows_of_that_round)
    first_task = rows_of_that_round.iloc[0]
    second_task = rows_of_that_round.iloc[1]
    third_task = rows_of_that_round.iloc[2]
    if len(rows_of_that_round) > 3:
        fourth_task = rows_of_that_round.iloc[3]

    if not math.isnan(first_task.rt):
        # respondimos durante la tone_number_task
        rt_tone_number_task = first_task.rt
        response_time_audio = rt_tone_number_task
        if second_task.task == 'delay_block':
            # entramos a delay->tone_number_task_remaining_before_delay (second y third tasks)
            if not math.isnan(third_task.rt):
                response_time_number = third_task.rt
            else:
                # no respondimos en remaining_before_delay, tenemos un empty block
                trial_duration_remaining_before_delay = 150.0 # el trial_duration de remaining_before_delay es siempre igual.
                response_time_number = fourth_task.rt + trial_duration_remaining_before_delay
        else:
            # second_task es 'remaining after delay'
            if not math.isnan(second_task.rt):
                response_time_number = second_task.rt + rt_tone_number_task - delay
            else: # vamos al empty block
                # no hace falta ver el trial_duration de la remaining_after_delay, porque ya tenemos el trial_duration de la tone_number_task:
                # tone_number_task.rt + remaining_after_delay.trial_duration = tone_number_task.trial_duration (ESTO SE CUMPLE SIEMPRE PORQUE ASÍ LO DEFINIMOS)
                response_time_number = third_task.rt + first_task.trial_duration - delay
    else:
        # no respondimos en la tone_number_task, tenemos dos empty blocks
        response_time_audio = second_task.rt + first_task.trial_duration
        response_time_number = third_task.rt + second_task.rt + first_task.trial_duration - delay


    return response_time_audio, response_time_number

In [ ]:
def response_times_for_df(df):
    """
    Retorna dos arreglos de arreglos de floats, indicando para cada ronda cuánto tardaron en responder a cada estímulo.
    El primer índice del arreglo que se retorna indica el bloque [0-3] y el segundo el número de ronda de ese bloque [0-39].
    """
    audio_response_times = [[None for _ in range(40)] for _ in range(4)]
    number_response_times = [[None for _ in range(40)] for _ in range(4)]
    for block_id in range(0, 4):
        for round_id in range(0, 40):
            audio_time, number_time = get_responses_time_of_block_and_round(df, block_id, round_id)
            audio_response_times[block_id][round_id] = audio_time
            number_response_times[block_id][round_id] = number_time
    return audio_response_times, number_response_times

In [ ]:
df_audio_rts, df_number_rts = response_times_for_df(df)
df_audio_rts_flat = [rt for block in df_audio_rts for rt in block]
df_number_rts_flat = [rt for block in df_number_rts for rt in block]

In [ ]:
# mean of a list of floats
def mean(lst):
    return sum(lst) / len(lst)

# mean of a list of floats, ignoring nan values. with only 2 decimal places.
def mean_ignore_nan(lst):
    lst_without_nans = [x for x in lst if not math.isnan(x)]
    return round(mean(lst_without_nans), 2)

In [ ]:
print(mean_ignore_nan(df_audio_rts_flat))
print(mean_ignore_nan(df_number_rts_flat))

¿No es raro que tarde casi lo mismo? Revisar bien el cálculo de los response times.